<h1>MINERAÇÃO DE DADOS HISTÓRICOS PARA IMPORTAÇÃO NO DAP E STORAGE !</h1>
<img src="https://tecnoblog.net/wp-content/uploads/2019/10/data-mining-700x394.jpg">

# Importações 

Importações necessárias para o projeto.

In [1]:
import os
import pandas as pd
import shutil

# Criando as pastas de trabalho

In [2]:
backup = input('Digite o nome da pasta backup: ')
scratch = input('crie um nome da pasta de scratch: ')
final = input('Crie o nome da pasta final: ')


Digite o nome da pasta backup: NOVO LOTE DAP
crie um nome da pasta de scratch: LoteDap3
Crie o nome da pasta final: Lote3DAP


<h3>Criando a pasta espelho</h3>

<p>Nesta pasta que vamos fazer a manipulação de arquivos, deixando pasta backup intacta</p>

In [3]:
origem = f'C:\\Nova pasta\\Documents\\{backup}'
destino = f'C:\\Nova pasta\\Documents\\{scratch}'

shutil.copytree(origem, destino)


'C:\\Nova pasta\\Documents\\LoteDap3'


# Deletando os arquivos já contidos no DAP

Com base nos arquivos de metadados xml, deletamos os arquivos com os mesmos nomes de todas as extensões

In [4]:
diretorio = f'C:\\Nova pasta\\Documents\\{scratch}'

count = 0

for root, dirs, files in os.walk(scratch):
    for arquivo in files:
        if arquivo.lower().endswith('.xml'):
            prefix = arquivo.split('.')[0]
            pasta = os.listdir(root)
            for arquivos in pasta:
                if prefix in arquivos:
                    count += 1
                    
                    
                    print(f'O arquvivo {arquivos} foi eliminado')
                    
                    
                                         
                    os.remove(f'{root}\\{arquivos}')
print(count)

0


# Criando um dataframe
Listando os arquivos da pasta em um dataframe contendo 3 colunas: Diretório, Aqrquivo e Tamanho do Arquivo

In [5]:
diretorio = f'C:\\Nova pasta\\Documents\\{scratch}'

df = pd.DataFrame(columns=list(['DIRETORIO', 'ARQUIVO', 'TAMANHO']))

for root, dirs, files in os.walk(diretorio):
    for file in files:
        tamanho = os.stat(f'{root}\\{file}').st_size
        linha = pd.DataFrame([[root, file, tamanho]], columns=list(['DIRETORIO', 'ARQUIVO', 'TAMANHO']))
        df = df.append(linha, ignore_index=True)

In [6]:
df.describe()

,DIRETORIO,ARQUIVO,TAMANHO
count,7858,7858,7858
unique,144,3325,1769
top,C:\Nova pasta\Documents\LoteDap3\Alvo_Formiga\...,_rangrid.con,59
freq,629,16,464


In [7]:
df.sort_values(['ARQUIVO', 'TAMANHO'])

,DIRETORIO,ARQUIVO,TAMANHO
565,C:\Nova pasta\Documents\LoteDap3\Alvo_Cutia\Ge...,00.CSV,3047
566,C:\Nova pasta\Documents\LoteDap3\Alvo_Cutia\Ge...,00NE.CSV,614
567,C:\Nova pasta\Documents\LoteDap3\Alvo_Cutia\Ge...,00NE.FLD,81238
568,C:\Nova pasta\Documents\LoteDap3\Alvo_Cutia\Ge...,00NE.IPM,33525
569,C:\Nova pasta\Documents\LoteDap3\Alvo_Cutia\Ge...,1000NE.CSV,3155
...,...,...,...
4690,C:\Nova pasta\Documents\LoteDap3\Alvo_Formiga\...,wizard.i3,1184
6459,C:\Nova pasta\Documents\LoteDap3\Alvo_Formiga\...,wizard.i3,1184
7410,C:\Nova pasta\Documents\LoteDap3\Alvo_Formiga\...,wizard.i3,1184
3754,C:\Nova pasta\Documents\LoteDap3\Alvo_FAUS\Geo...,wizard.i3,1616


# Eliminando duplicatas
Duplicatas de arquivos com nome e tamanho idênticos são eliminadas.

In [8]:
df_limpo = df.drop_duplicates(['ARQUIVO', 'TAMANHO'])

In [9]:
df_limpo.describe()

,DIRETORIO,ARQUIVO,TAMANHO
count,3583,3583,3583
unique,102,3325,1769
top,C:\Nova pasta\Documents\LoteDap3\Alvo_Cutia\Ge...,rangrid.log,59
freq,369,9,209


In [10]:
df_limpo[['ARQUIVO', 'TAMANHO']]

,ARQUIVO,TAMANHO
0,GCDBQV.mdb,2656256
1,SLC01FCC.SUT,372
2,SLC01FCD.SUT,2915
3,SLC02FCC.SUT,372
4,SLC02FCD.SUT,217
...,...,...
7853,_Z.itr,1620
7854,MI_200409_8534811993_Curionopolis_rpp.pdf,18934683
7855,Thumbs.db,18432
7856,TR_200210_geo_Projeto_Platina.pdf,7359856


# Criando a nova árvore e movendo os minerados
Aqui substituimos o o caminho e criamos toda a estrutura de diretórios, movendo os arquivos minerados contidos no dataframe "df_limpo" para a pasta vazia de destino.

In [11]:
for diretorio, arquivo in zip(df_limpo['DIRETORIO'],df_limpo['ARQUIVO']):
    diretorio_final = diretorio.replace(scratch, f'\\ToDAP\\{final}')
    try:
        caminho_carta = f'{diretorio_final}'
        if not os.path.exists(caminho_carta):
            os.makedirs(caminho_carta)
        shutil.copy(f'{diretorio}\\{arquivo}', f'{diretorio_final}\\{arquivo}')
    except:
        print(f'ERRO: {diretorio}\\{arquivo}')
        pass
    print(f'{diretorio_final}\\{arquivo}')

C:\Nova pasta\Documents\\ToDAP\Lote3DAP\Alvo_Cutia\Drilling\GCDBQV.mdb
C:\Nova pasta\Documents\\ToDAP\Lote3DAP\Alvo_Cutia\Drilling\Canal\SLC01FCC.SUT
C:\Nova pasta\Documents\\ToDAP\Lote3DAP\Alvo_Cutia\Drilling\Canal\SLC01FCD.SUT
C:\Nova pasta\Documents\\ToDAP\Lote3DAP\Alvo_Cutia\Drilling\Canal\SLC02FCC.SUT
C:\Nova pasta\Documents\\ToDAP\Lote3DAP\Alvo_Cutia\Drilling\Canal\SLC02FCD.SUT
C:\Nova pasta\Documents\\ToDAP\Lote3DAP\Alvo_Cutia\Drilling\Canal\SLC07PLC.SUT
C:\Nova pasta\Documents\\ToDAP\Lote3DAP\Alvo_Cutia\Drilling\Canal\SLC07PLD.SUT
C:\Nova pasta\Documents\\ToDAP\Lote3DAP\Alvo_Cutia\Drilling\Canal\SLC08PLC.SUT
C:\Nova pasta\Documents\\ToDAP\Lote3DAP\Alvo_Cutia\Drilling\Canal\SLC08PLD.SUT
C:\Nova pasta\Documents\\ToDAP\Lote3DAP\Alvo_Cutia\Drilling\Furos\A91-FD01\TTES8216.98
C:\Nova pasta\Documents\\ToDAP\Lote3DAP\Alvo_Cutia\Drilling\Furos\FD-01\SLT03FCC.SUT
C:\Nova pasta\Documents\\ToDAP\Lote3DAP\Alvo_Cutia\Drilling\Furos\FD-01\SLT03FCD.SUT
C:\Nova pasta\Documents\\ToDAP\Lote3DAP\

# Contador
Aqui podemos fazer uma consulta da quantidade de arquivos indicando uma pasta específica.
Os arquivos também são listados

In [12]:

conta = 0

diretorio = r'C:\Nova pasta\Documents\lote2'

for root, dirs, files in os.walk(scratch):
    for arquivo in files:
        if 'Geochem' in root:
            conta += 1
            print(arquivo)
        
        
print(conta)

0


In [63]:
origem = r'C:\Nova pasta\Documents\DAP'
destino = r'C:\Nova pasta\Documents\teste'

shutil.copytree(origem, destino)


'C:\\Nova pasta\\Documents\\teste'